# Add Images Of Objects To Trained Model (Creates New TF Records)

### You Should Still Be Able To Continue To Train From Your Last Checkpoint In Your Model

# 1. Import Dependencies

In [ ]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

##### 

# 2. Define Images to Collect

### Change Array To The Objects You Want To Add To Your Trained Model
### Change The Number Of Images You Would Like To Take Of Each Object

In [ ]:
labels = ['thumbsup', 'thumbsdown']
number_imgs = 20

##### 

# 3. Setup Folders 

### Adds Directories To Store Images Captured
### You Should Already Have These, But It Will Add Them If They Were Deleted

In [ ]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [ ]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
         !mkdir {IMAGES_PATH}
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

##### 

# 4. Capture Images

### Run This To Capture Images With Your Webcam
### It Will Run Through Each Element In The Array And Take The Amount Of Images Specified
### The Images Are Saved To Tensorflow > Workspace > Images > CollectedImages

In [ ]:
for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(2)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        cv2.setWindowProperty('frame', cv2.WND_PROP_TOPMOST, 1)
        cv2.moveWindow('frame', 400, 30)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

##### 

# 5. Image Labelling

### Run LabelImg Software To Annotate New Images

In [ ]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

### Runs The Image Labeling Software

In [ ]:
!cd {LABELIMG_PATH} && python labelImg.py

##### 

# 6. Use LabelImg Software To Annotate Images

### Make Sure To Use The Same Label(Case & Spelling) As You Did In Your Other Sets Of Images For Objects
### Use LabelImg To Annotate Object In Each Image 
### Same As In Notebook "1-Image-Collection" Step 6

##### 

# 7. Move Images And XML Files

### Move Your New Images And XML Files Into The Train And Test Folders
### Same As In Notebook "1-Image-Collection" Step 7

##### 

# 8. Create TF records (Overwrite With New Records)

### Recreates The TF Records With Your New Images Included
### Uses Your Annotated Images(Image and XML Files) You Saved Then Runs Them Through The Script

### Creates test.record And train.record File From Your Images and XML Files
### Located With The Label Map File At: Tensorflow > Workspace > Annotations

In [ ]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'
paths = {
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
 }

In [ ]:
files = {
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

##### 

# 9. You Are Now Ready To Continue To Train Your Model

### You Can Continue To Train From Your Last Checkpoint With The New Records At Notebook "Performance-Continue-To-Train-Model"